In [ ]:
import time
from collections import Counter

import cv2
import matplotlib.pyplot as plt
import numpy as np
import onnx
import onnxruntime as rt
import pandas as pd
from skl2onnx import to_onnx
from sklearn.neighbors import LocalOutlierFactor

In [ ]:
def print_inputs_outputs(session):
    for inp in session.get_inputs():
        print("INPUT:", inp)

    for out in session.get_outputs():
        print("OUTPUT:", out)

In [ ]:
model_folder = (
    "/path/to/model/youthful_mestorf_70e4c1e0/debug/onnx/youthful_mestorf_70e4c1e0.onnx"
)

out_model_path = "./youthful_mestorf_70e4c1e0_heatmap_ood.onnx"

train_df_file = "/path/to/model/youthful_mestorf_70e4c1e0/train.csv"
val_df_file = "/path/to/model/youthful_mestorf_70e4c1e0/val.csv"
test_df_file = "/path/to/model/youthful_mestorf_70e4c1e0/test.csv"

train_df = pd.read_csv(train_df_file)
val_df = pd.read_csv(val_df_file)
test_df = pd.read_csv(test_df_file)

In [ ]:
model = onnx.load(model_folder)
intermediate_tensor_name = (
    "model_1/youthful_mestorf_70e4c1e0/MobilenetV3small/avg_pool/Mean:0"
)
intermediate_layer_value_info = onnx.helper.ValueInfoProto()
intermediate_layer_value_info.name = intermediate_tensor_name
model.graph.output.extend([intermediate_layer_value_info])
onnx.save(model, "./gap.onnx")

In [ ]:
execution_provider = "CPUExecutionProvider"

gap_session = rt.InferenceSession("./gap.onnx", providers=[execution_provider])
print_inputs_outputs(gap_session)

In [ ]:
X_train = np.zeros((len(train_df), 576), dtype=np.float32)

for ix, img_fn in enumerate(train_df.filename.values):
    img = cv2.imread(img_fn, cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))

    img = np.array(np.expand_dims(img, axis=0), dtype=np.float32)
    outputs = gap_session.run(None, {"inputs": img})

    X_train[ix, :] = outputs[2][0, :, 0, 0]

In [ ]:
lof = LocalOutlierFactor(novelty=True)
lof.fit(X_train)

In [ ]:
lof_results = []

for ix, img_fn in enumerate(test_df.filename.values):
    img = cv2.imread(img_fn, cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))

    img = np.array(np.expand_dims(img, axis=0), dtype=np.float32)
    outputs = gap_session.run(None, {"inputs": img})

    feat_vec = outputs[2][0, :, 0, 0]
    pred = lof.predict(feat_vec.reshape(1, -1))[0]
    lof_results.append(pred)

In [ ]:
cnt = Counter(lof_results)
print(cnt)

In [ ]:
ix = np.argwhere(np.array(lof_results) == -1).flatten()
ood = test_df.iloc[ix].copy()

In [ ]:
new_img = cv2.imread(ood["filename"].iloc[1], cv2.IMREAD_COLOR)
new_img = cv2.cvtColor(new_img, cv2.COLOR_BGR2RGB)
new_img = cv2.resize(new_img, (224, 224))

plt.imshow(new_img)

In [ ]:
new_img = cv2.imread(
    "./image.png,
    cv2.IMREAD_COLOR,
)
new_img = cv2.cvtColor(new_img, cv2.COLOR_BGR2RGB)
new_img = cv2.resize(new_img, (224, 224))

img = np.array(np.expand_dims(new_img, axis=0), dtype=np.float32)
outputs = gap_session.run(None, {"inputs": img})

feat_vec = outputs[2][0, :, 0, 0]
pred = lof.predict(feat_vec.reshape(1, -1))[0]
print(pred)

plt.imshow(new_img)

In [ ]:
lof_onnx = to_onnx(lof, X_train[:1].astype(np.float32), target_opset=13)

In [ ]:
# lof_sess = rt.InferenceSession(lof_onnx.SerializeToString(), providers=["CPUExecutionProvider"])
# print_inputs_outputs(lof_sess)

In [ ]:
cnst = np.array([2, 3]).astype(np.int64)
node_cnst = onnx.helper.make_node(
    "Constant",
    inputs=[],
    outputs=["axes"],
    value=onnx.helper.make_tensor(
        name="axes",
        data_type=onnx.mapping.NP_TYPE_TO_TENSOR_TYPE[cnst.dtype],
        dims=cnst.shape,
        vals=cnst,
    ),
)

input_reshape = onnx.helper.make_node(
    "Squeeze",
    inputs=["x", "axes"],
    outputs=["y"],
)

In [ ]:
x = onnx.helper.make_tensor_value_info("x", onnx.TensorProto.FLOAT, [None, 576, 1, 1])
y = onnx.helper.make_tensor_value_info("y", onnx.TensorProto.FLOAT, [None, 576])

In [ ]:
graph = onnx.helper.make_graph(
    [node_cnst, input_reshape],  # nodes
    "reshape_input",  # a name
    inputs=[x],
    outputs=[y],
)

In [ ]:
reshape_model = onnx.helper.make_model(
    graph, opset_imports=[onnx.helper.make_operatorsetid("", 13)]
)
onnx.checker.check_model(reshape_model)

In [ ]:
reshape_model.ir_version = 7

In [ ]:
lof_reshaped_model = onnx.compose.merge_models(
    reshape_model, lof_onnx, io_map=[("y", "X")], outputs=["label", "scores"]
)

# onnx.save_model(lof_reshaped_model, "./lof_reshaped.onnx")

In [ ]:
# lof_reshaped_sess = rt.InferenceSession("./lof_reshaped.onnx", providers=["CPUExecutionProvider"])
# print_inputs_outputs(lof_reshaped_sess)

In [ ]:
combined_model = onnx.compose.merge_models(
    model,
    lof_reshaped_model,
    io_map=[
        ("model_1/youthful_mestorf_70e4c1e0/MobilenetV3small/avg_pool/Mean:0", "x")
    ],
    outputs=["youthful_mestorf_70e4c1e0", "tf.image.resize", "label", "scores"],
)

In [ ]:
onnx.checker.check_model(combined_model)
onnx.save_model(combined_model, out_model_path)

In [ ]:
comb_sess = rt.InferenceSession(out_model_path, providers=["CPUExecutionProvider"])

print_inputs_outputs(comb_sess)

In [ ]:
new_img = cv2.imread(ood["filename"].iloc[4], cv2.IMREAD_COLOR)
new_img = cv2.cvtColor(new_img, cv2.COLOR_BGR2RGB)
new_img = cv2.resize(new_img, (224, 224))

img = np.array(np.expand_dims(new_img, axis=0), dtype=np.float32)
st = time.time()
outputs = comb_sess.run(None, {"inputs": img})

print("Prediction time:", time.time() - st)
print("Label:", outputs[2][0])
print("Score:", outputs[3][0])

plt.imshow(new_img)

In [ ]:
new_img2 = new_img.copy()
new_img2[:, 75:100, :] = np.ones((224, 25, 3)) * 255

img = np.array(np.expand_dims(new_img2, axis=0), dtype=np.float32)
st = time.time()
outputs = comb_sess.run(None, {"inputs": img})

print("Prediction time:", time.time() - st)
print("Label:", outputs[2][0])
print("Score:", outputs[3][0])

plt.imshow(new_img2)

In [ ]:
test_imgs = np.zeros((5, 224, 224), dtype=np.int32)
heatmaps = np.zeros((5, 224, 224), dtype=np.float32)
labels, scores = [], []
pred = []

for i in range(5):
    new_img = cv2.imread(ood["filename"].iloc[i], cv2.IMREAD_COLOR)
    new_img = cv2.cvtColor(new_img, cv2.COLOR_BGR2RGB)
    new_img = cv2.resize(new_img, (224, 224))
    test_imgs[i, :] = cv2.cvtColor(new_img, cv2.COLOR_RGB2GRAY)
    img = np.array(np.expand_dims(new_img, axis=0), dtype=np.float32)
    outputs = comb_sess.run(None, {"inputs": img})
    heatmaps[i, :] = outputs[1][0, :, :, 0]

    pred.append(outputs[0][0])
    labels.append(outputs[2][0])
    scores.append(outputs[3][0])

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(16, 16))
fig.suptitle("Results 1")
axs[0][0].imshow(test_imgs[0, :])
axs[0][0].set_title(f"Label: {labels[0][0]}; Score:{scores[0][0]}")
axs[0][1].imshow(test_imgs[0, :])
axs[0][1].imshow(heatmaps[0, :], alpha=0.5, cmap="jet")
axs[0][1].set_title(f"Pred: {pred[0]}")

axs[1][0].imshow(test_imgs[1, :])
axs[1][0].set_title(f"Label: {labels[1][0]}; Score:{scores[1][0]}")
axs[1][1].imshow(test_imgs[1, :])
axs[1][1].imshow(heatmaps[1, :], alpha=0.5, cmap="jet")
axs[1][1].set_title(f"Pred: {pred[1]}")

In [ ]:
fig, axs = plt.subplots(3, 2, figsize=(20, 20))
fig.suptitle("Results 2")

axs[0][0].imshow(test_imgs[2, :])
axs[0][0].set_title(f"Label: {labels[2][0]}; Score:{scores[2][0]}")
axs[0][1].imshow(test_imgs[2, :])
axs[0][1].imshow(heatmaps[2, :], alpha=0.5, cmap="jet")
axs[0][1].set_title(f"Pred: {pred[2]}")

axs[1][0].imshow(test_imgs[3, :])
axs[1][0].set_title(f"Label: {labels[3][0]}; Score:{scores[3][0]}")
axs[1][1].imshow(test_imgs[3, :])
axs[1][1].imshow(heatmaps[3, :], alpha=0.5, cmap="jet")
axs[1][1].set_title(f"Pred: {pred[3]}")

axs[2][0].imshow(test_imgs[4, :])
axs[2][0].set_title(f"Label: {labels[4][0]}; Score:{scores[4][0]}")
axs[2][1].imshow(test_imgs[4, :])
axs[2][1].imshow(heatmaps[4, :], alpha=0.5, cmap="jet")
axs[2][1].set_title(f"Pred: {pred[4]}")

# Timing

In [ ]:
el_times = []

for ix, img_fn in enumerate(test_df.filename.values):
    img = cv2.imread(img_fn, cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    img = np.array(np.expand_dims(img, axis=0), dtype=np.float32)

    st = time.time()
    outputs = comb_sess.run(None, {"inputs": img})
    el_times.append(time.time() - st)

print("Mean prediction time on CPU:", np.mean(el_times))
print("Std prediction time on CPU:", np.std(el_times))

In [ ]:
comb_sess = rt.InferenceSession(out_model_path, providers=["CUDAExecutionProvider"])
print_inputs_outputs(comb_sess)

el_times = []

for ix, img_fn in enumerate(test_df.filename.values):
    img = cv2.imread(img_fn, cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    img = np.array(np.expand_dims(img, axis=0), dtype=np.float32)

    st = time.time()
    outputs = comb_sess.run(None, {"inputs": img})
    el_times.append(time.time() - st)

print("Mean prediction time on GPU:", np.mean(el_times))
print("Std prediction time on GPU:", np.std(el_times))

# END

In [ ]:
ad_sess = rt.InferenceSession(
    "/home/simone/workspace/anomaly-detection/output/flange/padim_keras/model.onnx",
    providers=["CPUExecutionProvider"],
)

print_inputs_outputs(ad_sess)

In [ ]:
el_times = []
for i in range(100):
    st = time.time()
    img = np.reshape(
        np.random.uniform(low=0, high=255, size=224 * 224 * 3), (1, 224, 224, 3)
    )
    outputs = ad_sess.run(None, {"inputs": np.array(img, dtype=np.float32)})

    el_times.append(time.time() - st)

print(np.mean(el_times))
print(np.std(el_times))